<a href="https://colab.research.google.com/github/RotemGat/NLP/blob/main/NLP_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set up

In [ ]:
!git clone https://github.com/NLP-Reichman/2025_assignment_1.git
!mv 2025_assignment_1/data data
!rm 2025_assignment_1/ -r

Cloning into '2025_assignment_1'...
remote: Enumerating objects: 71, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 71 (delta 24), reused 44 (delta 9), pack-reused 0 (from 0)
Receiving objects: 100% (71/71), 7.05 MiB | 6.59 MiB/s, done.
Resolving deltas: 100% (24/24), done.


# Introduction
In this assignment you will be creating tools for learning and testing language models. The corpora that you will be working with are lists of tweets in 8 different languages that use the Latin script. The data is provided either formatted as CSV or as JSON, for your convenience. The end goal is to write a set of tools that can detect the language of a given tweet.
The relevant files are under the data folder:

- en.csv (or the equivalent JSON file)
- es.csv (or the equivalent JSON file)
- fr.csv (or the equivalent JSON file)
- in.csv (or the equivalent JSON file)
- it.csv (or the equivalent JSON file)
- nl.csv (or the equivalent JSON file)
- pt.csv (or the equivalent JSON file)
- tl.csv (or the equivalent JSON file)

In [ ]:
import os
import json
from google.colab import files
import pandas as pd
import numpy as np
from itertools import product

# Implementation

## Part 1
Implement the function *preprocess* that iterates over all the data files and creates a single vocabulary, containing all the tokens in the data. Our token definition is a single UTF-8 encoded character. So, the vocabulary list is a simple Python list of all the characters that you see at least once in the data. The vocabulary should include the `<start>` and  `<end>` tokens.

Note - do NOT lowecase the sentences.

In [ ]:
def preprocess() -> list[str]:
  '''
  Return a list of characters, representing the shared vocabulary of all languages
  '''
  data_dir = "./data"
  vocab: set[str] = set()

  for fname in os.listdir(data_dir):
      if not fname.endswith(".csv"):
          continue  # skip JSON or other files

      csv_path = os.path.join(data_dir, fname)
      df = pd.read_csv(csv_path, usecols=["tweet_text"], encoding="utf-8")
      for tweet in df["tweet_text"].dropna():
          vocab.update(tweet)   # set.update() adds all characters at once
  vocab.update(["<start>", "<end>"])
  return list(vocab)

In [ ]:
vocab = preprocess()
print(f"vocab length: {len(vocab)}")
print(f"Some characters in the vocab: {vocab[:10]}")

vocab length: 1804
Some characters in the vocab: ['🙇', '機', '👇', '몬', '༼', '탄', '👠', '🌄', '🈶', '🎩']


## Part 2
Implement the function *build_lm* that generates a language model from a textual corpus. The function should return a dictionary (representing a model) where the keys are all the relevant *n*-1 sequences, and the values are dictionaries with the *n*_th tokens and their corresponding probabilities to occur. To ensure consistent probabilities calculation, please add n-1 `<start>` tokens to the beginning of a tweet and one `<end>` token at the end. For example, for a trigram model (tokens are characters), it should look something like:

{ "ab":{"c":0.5, "b":0.25, "d":0.25}, "ca":{"a":0.2, "b":0.7, "d":0.1} }

which means for example that after the sequence "ab", there is a 0.5 chance that "c" will appear, 0.25 for "b" to appear and 0.25 for "d" to appear.

Note - You should think how to add the add_one smoothing information to the dictionary and implement it.

Please add the `<unk>` token with $p(<unk>)=1|V|$ to the LM if buiulding a smoothed LM.

In [ ]:
def build_lm(lang: str, n: int, smoothed: bool = False) -> dict[str, dict[str, float]]:
  '''
  Return a language model for the given lang and n_gram (n)
  :param lang: the language of the model
  :param n: the n_gram value
  :param smoothed: boolean indicating whether to apply smoothing
  :return: a dictionary where the keys are n_grams and the values are dictionaries
  '''
  def load_tweets():
    # Load all tweets for the requested language
    csv_path  = f"./data/{lang}.csv"
    tweets = []
    if os.path.exists(csv_path):
        df = pd.read_csv(csv_path, encoding="utf-8")
        col = "tweet_text" if "tweet_text" in df.columns else "text"
        tweets = df[col].tolist()
    else:
        raise FileNotFoundError(f"No data found for language '{lang}'")
    return tweets

  def n_gram_counts(tweets):
    # Collect n‑gram counts:  context -> {next_token : count}
    counts: dict[str, dict[str, int]] = {}

    for tweet in tweets:
        tokens = ["<start>"] * (n - 1) + list(tweet) + ["<end>"]
        for i in range(len(tokens) - n + 1):
            context = "".join(tokens[i : i + n - 1])
            nxt     = tokens[i + n - 1]

            inner = counts.setdefault(context, {})
            inner[nxt] = inner.get(nxt, 0) + 1
    return counts

  def convert_counts_to_prob(counts):
      LM = {}
      for context, nxt_counts in counts.items():
          total = sum(nxt_counts.values())

          if not smoothed:
              LM[context] = {tok: c/total for tok, c in nxt_counts.items()}
              continue

          denom = total + V
          base  = {tok: (nxt_counts.get(tok, 0) + 1) / denom
                  for tok in pred_vocab}

          p_unk = 1.0 / V
          scale = 1.0 - p_unk
          LM[context] = {tok: p * scale for tok, p in base.items()}
          LM[context]["<unk>"] = p_unk

      if smoothed:
          LM["<unk>"] = {tok: 1.0 / V for tok in pred_vocab}

      return LM

  vocab = preprocess()
  pred_vocab = [tok for tok in vocab if tok != "<start>"]
  V = len(pred_vocab)

  tweets = load_tweets()
  counts = n_gram_counts(tweets)
  LM = convert_counts_to_prob(counts)

  return LM

In [ ]:
LM = build_lm("en", 3, False)
print(f"English Language Model with 3-gram is of length: {len(LM)}")

English Language Model with 3-gram is of length: 8238


## Part 3
Implement the function *eval* that returns the perplexity of a model (dictionary) running over the data file of the given target language.

The `<unk>` should be used for unknown contexts when calculating the perplexities.

In [ ]:
def perplexity(model: dict, text: list, n) -> float:
  '''
  Calculates the perplexity of the given string using the given language model.
  :param model: The language model
  :param text: The tokenized text to calculate the perplexity for
  :param n: The n-gram of the model
  :return: The perplexity
  '''
  # padding
  tokens = ["<start>"] * (n - 1) + text + ["<end>"]

  # vocabulary size (|V|) — ignore the <unk> entry if present
  sample_ctx = next(iter(model.values()))
  V = len([t for t in sample_ctx if t != "<unk>"])

  # uniform back‑off prob if we cannot find <unk>
  uniform_p = 1.0 / max(V, 1)

  log_prob_sum = 0.0
  steps = 0

  # slide an n‑length window
  for i in range(n - 1, len(tokens)):
      context = "".join(tokens[i - n + 1 : i])  # n‑1 chars
      nxt     = tokens[i]

      # retrieve probability

      if context in model:
          prob = model[context].get(nxt,
                      model[context].get("<unk>", uniform_p))
      else:
          prob = uniform_p

      prob = max(prob, 1e-12)

      log_prob_sum += np.log(prob)
      steps += 1

  # average log‑probability to perplexity
  return np.exp(-log_prob_sum / steps)

In [ ]:
def eval(model: dict, target_lang: str, n: int) -> float:
  '''
  Return the perplexity value calculated over applying the model on the text file
  of the target_lang language.
  :param model: the language model
  :param target_lang: the target language
  :param n: The n-gram of the model
  :return: the perplexity value
  '''
  csv_path  = f"./data/{target_lang}.csv"

  if os.path.exists(csv_path):
      df  = pd.read_csv(csv_path, encoding="utf-8")
      col = "tweet_text" if "tweet_text" in df.columns else "text"
      tweets = df[col].dropna().tolist()
  else:
      raise FileNotFoundError(f"No data for language '{target_lang}'")

  # accumulate log‑probability via the helper
  log_prob_sum = 0.0
  token_count  = 0

  for tweet in tweets:
      char_list = list(tweet)                # no padding here
      sent_pp   = perplexity(model, char_list, n)

      # perplexity definition:  PP = exp(‑logP / L)  ⇒  logP = ‑L·log(PP)
      L = len(char_list) + 1                 # +1 for <end> (padding handled inside)
      log_prob_sum -= L * np.log(sent_pp)
      token_count  += L

  # corpus perplexity
  return float(np.exp(-log_prob_sum / token_count))

In [ ]:
LM = build_lm("en", 3, True)

In [ ]:
print("Perplexity of the English 3-gram model on datasets:")
print(f"On English: {eval(LM, 'en', 3): .2f}")
print(f"On French: {eval(LM, 'fr', 3): .2f}")
print(f"On Dutch: {eval(LM, 'nl', 3): .2f}")
print(f"On Tagalog: {eval(LM, 'tl', 3): .2f}")


Perplexity of the English 3-gram model on datasets:
On English:  26.75
On French:  58.50
On Dutch:  61.42
On Tagalog:  70.61


In [ ]:
lm1 = build_lm("en", 1, True)
lm2 = build_lm("en", 2, True)
lm3 = build_lm("en", 3, True)
lm4 = build_lm("en", 4, True)

print("Perplexity on differnet n-gram models on English")
print(f"On 1-gram: {eval(lm1, 'en', 1): .2f}")
print(f"On 2-gram: {eval(lm2, 'en', 2): .2f}")
print(f"On 3-gram: {eval(lm3, 'en', 3): .2f}")
print(f"On 4-gram: {eval(lm4, 'en', 4): .2f}")

Perplexity on differnet n-gram models on English
On 1-gram:  37.88
On 2-gram:  21.31
On 3-gram:  26.75
On 4-gram:  59.88


## Part 4
Implement the *match* function that calls *eval* using a specific value of *n* for every possible language pair among the languages we have data for. You should call *eval* for every language pair four times, with each call assign a different value for *n* (1-4). Each language pair is composed of the source language and the target language. Before you make the call, you need to call the *lm* function to create the language model for the source language. Then you can call *eval* with the language model and the target language. The function should return a pandas DataFrame with the following four columns: *source_lang*, *target_lang*, *n*, *perplexity*. The values for the first two columns are the two-letter language codes. The value for *n* is the *n* you use for generating the specific perplexity values which you should store in the forth column.

In [ ]:
languages = ['en', 'es', 'fr', 'in', 'it', 'nl', 'pt', 'tl']

In [ ]:
def match() -> pd.DataFrame:
  '''
  Return a DataFrame containing one line per every language pair and n_gram.
  Each line will contain the perplexity calculated when applying the language model
  of the source language on the text of the target language.
  :return: a DataFrame containing the perplexity values
  '''
  rows = []
  for src in languages:
      for n in range(1, 5):
          lm = build_lm(src, n, smoothed=True)

          for tgt in languages:
              pp = eval(lm, tgt, n)
              rows.append({
                  "source": src,
                  "target": tgt,
                  "n":           n,
                  "perplexity":  pp
              })

  return pd.DataFrame(rows)

## Part 5
Implement the *generate* function which takes a language code, *n*, the prompt (the starting text), the number of tokens to generate, and *r*, which is the random seed for any randomized action you plan to take in your implementation. The function should start generating tokens, one by one, using the language model of the given source language and *n*. The prompt should be used as a starting point for aligning on the probabilities to be used for generating the next token.

Note - The generation of the next token should be from the LM's distribution with NO smoothing.

In [ ]:
def generate(lang: str, n: int, prompt: str, number_of_tokens: int, r: int) -> str:
  '''
  Generate text in the given language using the given parameters.
  :param lang: the language of the model
  :param n: the n_gram value
  :param prompt: the prompt to start the generation
  :param number_of_tokens: the number of tokens to generate
  :param r: the random seed to use
  '''
  np.random.seed(r)
  lm = build_lm(lang, n, smoothed=False)            # unsmoothed LM

  text = list(prompt)                               # mutable buffer
  start_ctx = "<start>" * (n - 1)                   # fallback context
  vocab = np.array(sorted({t for d in lm.values() for t in d}))  # for uniform fallback

  for _ in range(number_of_tokens):
      ctx = "".join(text[-(n - 1):]) if n > 1 else ""            # last n‑1 chars
      dist = lm.get(ctx) or lm.get(start_ctx)                    # back‑off
      if dist:                                                   # normal case
          tokens, probs = zip(*dist.items())
          token = np.random.choice(tokens, p=probs)
      else:                                                      # unseen everywhere → uniform
          token = np.random.choice(vocab)

      if token == "<end>": break
      text.append(token)

  return "".join(text)

## Part 6
Play with your generate function, try to generate different texts in different language and various values of *n*. No need to submit anything of that.

In [ ]:
print(generate('en', 1, "I am", 10, 5))
print(generate('en', 2, "I am", 10, 5))
print(generate('en', 3, "I am", 10, 5))
print(generate('en', 4, "I am ", 10, 5))
print(generate('es', 2, "Soy", 10, 5))
print(generate('es', 3, "Soy", 10, 5))
print(generate('fr', 2, "Je suis", 10, 5))
print(generate('fr', 3, "Je suis", 10, 5))

I amD.lIntudyE
I amell1Gige o
I amebompirs a
I am surrance t
Soy ' Dusos q
Soy ensanta o
Je suis jeitpu s 
Je suistreil Her 


# Testing

Copy the content of the **tests.py** file from the repo and paste below. This will create the results.json file and download it to your machine.

In [ ]:
########################################
# PLACE TESTS HERE #
# Create tests
def test_preprocess():
    return {
        'vocab_length': len(preprocess()),
    }

def test_build_lm():
    return {
        'english_2_gram_length': len(build_lm('en', 2, True)),
        'english_3_gram_length': len(build_lm('en', 3, True)),
        'french_3_gram_length': len(build_lm('fr', 3, True)),
        'spanish_3_gram_length': len(build_lm('es', 3, True)),
    }

def test_eval():
    lm = build_lm('en', 3, True)
    return {
        'en_on_en': round(eval(lm, 'en', 3), 2),
        'en_on_fr': round(eval(lm, 'fr', 3), 2),
        'en_on_tl': round(eval(lm, 'tl', 3), 2),
        'en_on_nl': round(eval(lm, 'nl', 3), 2),
    }

def test_match():
    df = match()
    return {
        'df_shape': df.shape,
        'en_en_3': df[(df['source'] == 'en') & (df['target'] == 'en') & (df['n'] == 3)]['perplexity'].values[0],
        'en_tl_3': df[(df['source'] == 'en') & (df['target'] == 'tl') & (df['n'] == 3)]['perplexity'].values[0],
        'en_nl_3': df[(df['source'] == 'en') & (df['target'] == 'nl') & (df['n'] == 3)]['perplexity'].values[0],
    }

def test_generate():
    return {
        'english_2_gram': generate('en', 2, "I am", 20, 5),
        'english_3_gram': generate('en', 3, "I am", 20, 5),
        'english_4_gram': generate('en', 4, "I Love", 20, 5),
        'spanish_2_gram': generate('es', 2, "Soy", 20, 5),
        'spanish_3_gram': generate('es', 3, "Soy", 20, 5),
        'french_2_gram': generate('fr', 2, "Je suis", 20, 5),
        'french_3_gram': generate('fr', 3, "Je suis", 20, 5),
    }

TESTS = [test_preprocess, test_build_lm, test_eval, test_match, test_generate]

# Run tests and save results
res = {}
for test in TESTS:
    try:
        cur_res = test()
        res.update({test.__name__: cur_res})
    except Exception as e:
        res.update({test.__name__: repr(e)})

with open('results.json', 'w') as f:
    json.dump(res, f, indent=2)

# Download the results.json file
files.download('results.json')
########################################

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Show the local files, results.json should be there now and
# also downloaded to your local machine
!ls -l